The quality of the data and the amount of useful information that it contains are key
factors that determine how well a machine learning algorithm can learn. Therefore,
it is absolutely critical to ensure that we examine and preprocess a dataset before
we feed it to a learning algorithm. 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [209]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

sns.set_style()

In [29]:
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [30]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [31]:
# accessing the numpy array from a dataframe object
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

###  Dropping missing values

In [32]:
# dropping missing values across the rows
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [33]:
# dropping missing values down each columns
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [39]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


disadvantages; for example, we may end up removing too
many samples, which will make a reliable analysis impossible. Or, if we remove too
many feature columns, we will run the risk of losing valuable information that our
classifier needs to discriminate between classes

###  Filling Missing values
This can be done with the pandas **`fillna()`** method or using scikit-learn's **`imputer`** class

In [61]:
# Using SimpleImputer() method
imputed_data = SimpleImputer(strategy='mean').fit_transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [63]:
# using fillna() method
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### Handling Categorical Values

In [73]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']])
df.columns = ['color', 'size', 'price', 'classlabel']

In [74]:
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


#### Mapping Function

In [77]:
size_mapping = {
    'XL':3, 'M':1, 'L':2
}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [98]:
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}

df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [112]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [128]:
# Using OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

# Partitioning a dataset into separate training and test datasets

In [ ]:
file = '../data/wine/wine.data'
names = ['Class label', 'Alcohol','Malic acid', 'Ash','Alcalinity of ash', 'Magnesium','Total phenols', 'Flavanoids',
'Nonflavanoid phenols','Proanthocyanins','Color intensity', 'Hue','OD280/OD315 of diluted wines','Proline']

df_wine = pd.read_csv(file, names=names, header=None)

In [164]:
df_wine.head()

,class label,alcohol,malic acid,ash,alcalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,od280/od315 of diluted wines,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [170]:
print('Class Label');df_wine['class label'].unique()

Class Label

array([1, 2, 3])

In [184]:
# making features and label columns
X, y = df_wine.iloc[:, 1:], df_wine.iloc[:, 0]
y.shape, X.shape

((178,), (178, 13))

In [194]:
# making Partition 
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((54, 13), (54,), (124, 13), (124,))

#  Feature Scaling
Scale both datasets together, but do not fit the scaling object to the test dataset.

In [258]:
# Using min-max scaling
mms = MinMaxScaler()
X_train_scaled = mms.fit_transform(X_train)
X_test_scaled = mms.transform(X_test)

In [259]:
std = StandardScaler()
X_train_scaled = std.fit_transform(X_train)
X_test_scaled = std.transform(X_test)

X_train_scaled[:3, :2], X_test_scaled[:3, :2]

(array([[ 0.71225893,  2.22048673],
        [ 0.88229214, -0.70457155],
        [ 0.84585645, -0.73022996]]),
 array([[ 0.89443737, -0.38811788],
        [-1.04879931, -0.77299397],
        [ 1.64744158, -0.41377628]]))